In [23]:
import torch
import platform
platform = platform.system()

In [24]:
if platform == 'Linux':
    !BASICSR_JIT=True
    pretrained_hqp_path = "../pretrained_models/pretrained_HQPs_New.pth"
    pretrained_net_path = "../pretrained_models/pretrained_RIDCP_New.pth"
else:
    !set BASICSR_JIT=True
    pretrained_hqp_path = "../pretrained_models/pretrained_HQPs_New.pth"
    pretrained_net_path = "../pretrained_models/pretrained_RIDCP_New.pth"

In [25]:
param_key = 'params'
load_hqp = torch.load(pretrained_hqp_path)[param_key]
load_net = torch.load(pretrained_net_path)[param_key]

In [26]:
for key, param in load_hqp.items():
    print(key)

multiscale_encoder.in_conv.weight
multiscale_encoder.in_conv.bias
multiscale_encoder.blocks.0.0.weight
multiscale_encoder.blocks.0.0.bias
multiscale_encoder.blocks.0.1.conv.0.norm.weight
multiscale_encoder.blocks.0.1.conv.0.norm.bias
multiscale_encoder.blocks.0.1.conv.2.weight
multiscale_encoder.blocks.0.1.conv.2.bias
multiscale_encoder.blocks.0.1.conv.3.norm.weight
multiscale_encoder.blocks.0.1.conv.3.norm.bias
multiscale_encoder.blocks.0.1.conv.5.weight
multiscale_encoder.blocks.0.1.conv.5.bias
multiscale_encoder.blocks.0.2.conv.0.norm.weight
multiscale_encoder.blocks.0.2.conv.0.norm.bias
multiscale_encoder.blocks.0.2.conv.2.weight
multiscale_encoder.blocks.0.2.conv.2.bias
multiscale_encoder.blocks.0.2.conv.3.norm.weight
multiscale_encoder.blocks.0.2.conv.3.norm.bias
multiscale_encoder.blocks.0.2.conv.5.weight
multiscale_encoder.blocks.0.2.conv.5.bias
multiscale_encoder.blocks.1.0.weight
multiscale_encoder.blocks.1.0.bias
multiscale_encoder.blocks.1.1.conv.0.norm.weight
multiscale_en

In [27]:
for key, param in load_net.items():
    print(key)

multiscale_encoder.in_conv.weight
multiscale_encoder.in_conv.bias
multiscale_encoder.blocks.0.0.weight
multiscale_encoder.blocks.0.0.bias
multiscale_encoder.blocks.0.1.conv.0.norm.weight
multiscale_encoder.blocks.0.1.conv.0.norm.bias
multiscale_encoder.blocks.0.1.conv.2.weight
multiscale_encoder.blocks.0.1.conv.2.bias
multiscale_encoder.blocks.0.1.conv.3.norm.weight
multiscale_encoder.blocks.0.1.conv.3.norm.bias
multiscale_encoder.blocks.0.1.conv.5.weight
multiscale_encoder.blocks.0.1.conv.5.bias
multiscale_encoder.blocks.0.2.conv.0.norm.weight
multiscale_encoder.blocks.0.2.conv.0.norm.bias
multiscale_encoder.blocks.0.2.conv.2.weight
multiscale_encoder.blocks.0.2.conv.2.bias
multiscale_encoder.blocks.0.2.conv.3.norm.weight
multiscale_encoder.blocks.0.2.conv.3.norm.bias
multiscale_encoder.blocks.0.2.conv.5.weight
multiscale_encoder.blocks.0.2.conv.5.bias
multiscale_encoder.blocks.1.0.weight
multiscale_encoder.blocks.1.0.bias
multiscale_encoder.blocks.1.1.conv.0.norm.weight
multiscale_en

In [28]:
from basicsr.archs.ridcp_new_arch import RIDCPNew

net = RIDCPNew(LQ_stage=False)
hqp = RIDCPNew(LQ_stage=True)

In [29]:
ridcp_key = set(net.state_dict().keys())
load_ridcp_key = set(load_net.keys())

In [30]:
ridcp_key == load_ridcp_key

False

In [31]:
hqp_key = set(hqp.state_dict())
load_hqp_key = set(load_net.keys())

In [32]:
load_hqp_key == hqp_key

False

In [33]:
new_hqp = {}
for key, value in load_hqp.items():
    if key.startswith('multiscale_encoder.blocks.2.swin_blks.'):
        new_key = key.replace('multiscale_encoder.blocks.2.swin_blks.', 'ridcp_encoder.swin_blks.', 1)
    elif key.startswith('multiscale_encoder.'):
        new_key = key.replace('multiscale_encoder.', 'vq_encoder.', 1)
    elif key.startswith('decoder_group.'):
        new_key = key.replace('decoder_group.', 'vq_decoder_group.', 1)
    elif key.startswith('multiscale_decoder.'):
        new_key = key.replace('multiscale_decoder.', 'ridcp_decoder.', 1)
    else:
        new_key = key
    new_hqp[new_key] = value

new_net = {}
for key, value in load_net.items():
    if key.startswith('multiscale_encoder.blocks.2.swin_blks.'):
        new_key = key.replace('multiscale_encoder.blocks.2.swin_blks.', 'ridcp_encoder.swin_blks.', 1)
    elif key.startswith('multiscale_encoder.'):
        new_key = key.replace('multiscale_encoder.', 'vq_encoder.', 1)
    elif key.startswith('decoder_group.'):
        new_key = key.replace('decoder_group.', 'vq_decoder_group.', 1)
    elif key.startswith('multiscale_decoder.'):
        new_key = key.replace('multiscale_decoder.', 'ridcp_decoder.', 1)
    else:
        new_key = key
    new_net[new_key] = value

In [34]:
hqp_state = hqp.state_dict()
if set(hqp_state.keys()) == set(new_hqp.keys()):
    hqp_state.update(new_hqp)
    hqp.load_state_dict(hqp_state)
    save_dict = {param_key: hqp.state_dict()}
    # torch.save(save_dict, "../pretrained_models/pretrained_HQPs_renamed.pth")
else:
    print(set(hqp_state.keys()) - set(new_hqp.keys()))
    print("-------------------------------")
    print(set(new_hqp.keys()) - set(hqp_state.keys()))
    
net_state = net.state_dict()
if set(net_state.keys()) == set(new_net.keys()):
    net_state.update(new_net)
    hqp.load_state_dict(net_state)
    save_dict = {param_key: hqp.state_dict()}
    # torch.save(save_dict, "../pretrained_models/pretrained_RIDCP_renamed.pth")
else:
    print(set(net_state.keys()) - set(new_net.keys()))
    print("-------------------------------")
    print(set(new_net.keys()) - set(net_state.keys()))

{'ridcp_encoder.swin_blks.0.residual_group.blocks.1.norm2.weight', 'ridcp_encoder.swin_blks.0.residual_group.blocks.3.attn.proj.bias', 'ridcp_encoder.swin_blks.2.residual_group.blocks.3.mlp.fc2.weight', 'ridcp_encoder.swin_blks.1.residual_group.blocks.5.attn_mask', 'ridcp_encoder.swin_blks.1.residual_group.blocks.5.norm2.weight', 'ridcp_encoder.swin_blks.1.residual_group.blocks.1.mlp.fc1.bias', 'ridcp_encoder.swin_blks.0.residual_group.blocks.1.norm1.bias', 'ridcp_encoder.swin_blks.1.residual_group.blocks.5.attn.relative_position_bias_table', 'ridcp_decoder.upsampler.1.3.conv.2.bias', 'ridcp_encoder.swin_blks.1.residual_group.blocks.2.attn.proj.bias', 'ridcp_encoder.swin_blks.2.residual_group.blocks.3.mlp.fc1.bias', 'ridcp_encoder.swin_blks.1.residual_group.blocks.5.mlp.fc1.bias', 'ridcp_encoder.swin_blks.2.residual_group.blocks.2.norm1.weight', 'ridcp_encoder.swin_blks.1.residual_group.blocks.3.norm1.bias', 'ridcp_encoder.swin_blks.3.residual_group.blocks.4.norm1.weight', 'ridcp_encod

In [ ]:
prefix = 'feature_extract.'
# 使用字典推导式为每个键添加前缀
load_net = {prefix + key: value for key, value in load_net.items()}